<a href="https://colab.research.google.com/github/PanitnanKan/BADS7105-CRM-analysis-and-intelligence/blob/main/Homework%2003/CRM_Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pythainlp
!pip install langdetect
!pip install keras
#!pip install deepcut
#!pip install tensorflow
#!pip install langdetect

     |████████████████████████████████| 10.6MB 8.0MB/s 
     |████████████████████████████████| 747kB 46.9MB/s 
     |████████████████████████████████| 983kB 7.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=20b9fb91d487c60fc648adb986dae900cf42c3c0c16cd404cfe151e2a25f01d7
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [5]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#from dateutil.relativedelta import relativedelta
from datetime import datetime,date
import pythainlp
from pythainlp.tokenize import word_tokenize
from langdetect import detect
#import deepcut

In [6]:
activities_all = pd.read_excel('Activities_all.xlsx')
activities_all.head()


,ID,Time,Activity,Pain,Gain
0,6210422010,06:30:00,ตื่นนอน อาบน้ำ แปรงฟัน,ง่วง,ตื่นเช้า ประปรี้ประเปร่า
1,6210422010,07:30:00,ขับรถไปทำงาน,รถติด,กินข้าวฟังเพลงบนรถ
2,6210422010,08:30:00,ถึงที่ทำงาน,คล้ายจะขี้เกียจทำงาน,ได้ที่จอดรถในชั้นที่เดินเข้าตึกได้ง่าย
3,6210422010,08:45:00,เดินไปร้านกาแฟ,ช่วงเวลาคนเยอะ ต่อคิวนาน,เจอคู่รักใหม่มาแอบจิบกาแฟ ได้ฟังเรื่องชาวบ้า...
4,6210422010,09:30:00,ประชุมกับผู้บริหาร,หาข้อสรุปไม่ได้,ทุกคนมีข้อเสนอแนะเยอะมาก


In [7]:
activities_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        892 non-null    int64 
 1   Time      892 non-null    object
 2   Activity  892 non-null    object
 3   Pain      706 non-null    object
 4   Gain      761 non-null    object
dtypes: int64(1), object(4)
memory usage: 35.0+ KB


In [8]:
lg_list = []
#activities_all = activities_all['token']
for index,row in activities_all.iterrows():
  sen = row['Activity'] 
  lg = detect(sen)
  lg_list.append(lg)
activities_all['lang'] = lg_list

token_list = []
for index,row in activities_all.iterrows():
  sen = row['Activity'] 
  if  row['lang'] == 'th':
    tokens = word_tokenize(sen, engine='longest',keep_whitespace=False)
    #print(tokens)
    token_list.append(tokens)
  else:
    tokens = sen.split()
    tokens_lower =[]
    for text in tokens:
      texts = text.lower ()
      tokens_lower.append(texts)
    token_list.append(tokens_lower)
     

activities_all['token'] = token_list

activities_all

,ID,Time,Activity,Pain,Gain,lang,token
0,6210422010,06:30:00,ตื่นนอน อาบน้ำ แปรงฟัน,ง่วง,ตื่นเช้า ประปรี้ประเปร่า,th,"[ตื่นนอน, อาบน้ำ, แปรงฟัน]"
1,6210422010,07:30:00,ขับรถไปทำงาน,รถติด,กินข้าวฟังเพลงบนรถ,th,"[ขับรถ, ไป, ทำงาน]"
2,6210422010,08:30:00,ถึงที่ทำงาน,คล้ายจะขี้เกียจทำงาน,ได้ที่จอดรถในชั้นที่เดินเข้าตึกได้ง่าย,th,"[ถึงที่, ทำงาน]"
3,6210422010,08:45:00,เดินไปร้านกาแฟ,ช่วงเวลาคนเยอะ ต่อคิวนาน,เจอคู่รักใหม่มาแอบจิบกาแฟ ได้ฟังเรื่องชาวบ้า...,th,"[เดิน, ไป, ร้านกาแฟ]"
4,6210422010,09:30:00,ประชุมกับผู้บริหาร,หาข้อสรุปไม่ได้,ทุกคนมีข้อเสนอแนะเยอะมาก,th,"[ประชุม, กับ, ผู้บริหาร]"
...,...,...,...,...,...,...,...
887,6310422093,19:00:00,Fitness,Tired,Burn calories,de,[fitness]
888,6310422093,20:30:00,Drive a car to home,Bad traffic and less vision,Safty arrive,en,"[drive, a, car, to, home]"
889,6310422093,21:30:00,Take a shower,Concentrate body washing,Get clean,en,"[take, a, shower]"
890,6310422093,22:00:00,Gaming on computer,Wasted time,Entertaining,en,"[gaming, on, computer]"


In [57]:
activity_type = []
df = pd.DataFrame(columns=['ID','Time','Activity','Type','Pain','Gain'])
for index,row in activities_all.iterrows():
  data_list = []
  temp =[]
  atv_list = row['token']
  for text in row['token']:
    if text in ['ตื่นนอน','ตื่น','นาฬิกาปลุก','ปลุก','wake']: 
      a_type = 'ตื่นนอน' 
    elif text in ['อาบน้ำ','ล้างหน้า','shower','bath']:
      a_type = 'อาบน้ำ' 
    elif text in ['แต่งตัว','ครีม','แต่งหน้า','skin','เตรียมตัว','ชุด','preparing','dress']:
      a_type = 'แต่งตัว' 
    elif (text in ['เดินทางออก','กลับบ้าน','ขับรถ','เดินทาง','drive','van','bts','mrt','BTS','MRT', 'รถไฟฟ้า', 'บีทีเอส', 'กลับ','back','ออกจาก','move','ออก','go','going','มอเตอร์','วิน','ขึ้นรถ','แท๊กซี่','taxi','bus','out','เดิน']):
     if 'เล่น' not in atv_list and  'ดู' not in atv_list and 'sleep' not in atv_list and 'bed' not in atv_list and 'preparing' not in atv_list:
     #and (('เล่น'or 'ดู' or 'sleep' or 'bed' or 'preparing') not in row['token']) : 
        a_type = 'เดินทาง'
    elif text in ['เข้างาน','email','อีเมล','แผนก','po','อีเมลล์','ประชุม','ทำงาน','work','meeting','working','งาน','ทำงานต่อ','การทำงาน','อีเมล์','mail','user','ลูกค้า','morning','สกรัม','เอกสาร','requirement','เข้าทำงาน','การผลิต','พอร์ต']:
      if 'ไป' not in atv_list and 'เตรียมตัว' not in atv_list and 'แต่งตัว' not in atv_list and 'go' not in atv_list and 'going'not in atv_list and 'ถึงที่'not in atv_list and  'กินข้าว'not in atv_list :
        a_type = 'ทำงาน'
    elif (text in ['สั่ง','อาหาร','delivery','ของกิน','order','สั่งอาหาร']) :
      if 'line' in atv_list or 'grab' in atv_list or 'grabfood' in atv_list or 'panda' in atv_list :
      #and (('line' or 'grab' or 'grabfood' or 'panda' ) in atv_list):
        a_type = 'สั่งอาหาร Delivery'
    elif (text in ['พักเที่ยง','รับประทาน','กินข้าว','อาหาร','ทานอาหาร','อาหารว่าง','กิน','ทานข้าว','breakfast','lunch','dinner','ทาน','สั่งอาหาร','ข้าวกลางวัน']) :
      if 'ขนม' not in atv_list and 'ของว่าง' not in atv_list and 'กาแฟ' not in atv_list and 'ซื้อ' not in atv_list:
    #and (('ขนม' or 'ของว่าง' or 'กาแฟ' or 'ซื้อ' ) not in atv_list):
        a_type = 'ทานอาหาร'
    elif text in ['horoscope','มือถือ','podcast','ถ่ายรูป','คลิป','อินเตอร์เน็ต','social','facebook','อัพรูป','post','Internet','internet','instragram','instagram','อินเตอร์','โทรศัพท์','twitter', 'บทความ','phone']:
      a_type = 'เล่น Social'
    elif (text in ['youtube','netflix','หนัง','ซี','movie','series','tv','ทีวี','นิยาย','การ์ตูน','บันเทิง','ฟังเพลง']) and (('content' or 'โทรศัพท์' ) not in atv_list): 
      a_type = 'Entertainment'  
    elif text in ['game','Game','gaming','เกม','เกมส์','rov','games']:
      a_type = 'เล่นเกมส์'
    elif (text in ['ซื้อ','ซื้อของ','shopping','shopee','lazada','ไปหา']) :
      if 'ขนม' not in atv_list and 'ของกิน' not in atv_list and 'ชา' not in atv_list and 'กับข้าว' not in atv_list and 'กาแฟ' not in atv_list and 'น้ำผลไม้' not in atv_list and 'อาหาร' not in atv_list and 'น้ำ' not in atv_list and 'น้ำหวาน' not in atv_list :
    #and (('ขนม' or 'ของกิน' or 'ชา' or 'กับข้าว' or 'กาแฟ' or 'น้ำผลไม้' or 'อาหาร' or 'น้ำ' or 'น้ำหวาน' ) not in atv_list):
        a_type = 'Shopping'
    elif (text in ['ซื้อ','ซื้อของ','shopping','สั่ง','ไปหา']) : 
      if 'ขนม' in atv_list or 'ของกิน' in atv_list or 'ชา' in atv_list or 'กับข้าว' in atv_list or 'กาแฟ' in atv_list or 'น้ำผลไม้' in atv_list or 'อาหาร' in atv_list or 'น้ำ' or 'ข้าวเย็น' in atv_list or 'ข้าวเหนียว' in atv_list:
        a_type = 'ซื้อของกิน'
    elif (text in ['ร้านกาแฟ','กาแฟ','coffee','ชา','น้ำผลไม้','ชงกาแฟ']) and ('ซื้อ' not in atv_list):
      a_type = 'ดื่มเครื่องดื่มเพิ่มพลัง'
    elif (text in ['เดินเล่น','คาเฟ่','ริมทะเล','โรงแรม','เที่ยว','ห้างสรรพสินค้า']) and (('ตอน') not in atv_list) :
      a_type = 'ท่องเที่ยว'
    elif (text in ['girl','friend','ครอบครัว','ญาติ','family','พ่อ','แม่','พี่','น้อง','เพื่อนร่วมงาน','ทีม','พี่สาว','เพื่อน','บ้าน','ทีมงาน']) :
      if 'คุย' in atv_list or 'พูดคุย' in atv_list or 'call' in atv_list or 'เม้าท์' in atv_list or 'time' in atv_list :
    #and (( 'คุย' or'พูดคุย' or 'call' or 'เม้าท์') in atv_list):
        a_type = 'สร้างความสัมพันธ์'  
    elif (text in ['sandwich','อาหารว่าง','ตู้เย็น','ขนม','break','เบรค','รองท้อง','พักผ่อน','งีบหลับ','นวด','งีบ']) and (('ซื้อ') not in atv_list):
      a_type = 'พักเบรค พักผ่อน'   
    elif (text in ['แมว','หมา','cat','dog','สุนัข']) and (('ซื้อ') not in atv_list):
      a_type = 'Pet'    
    elif (text in ['quiz','class','เรียนหนังสือ','เรียน','study','learn','การบ้าน', 'homework','learning','read','หนังสือ','อ่านหนังสือ','ทำการบ้าน','research']) and (('นิ'or'ยาย'or'การ์ตูน'or'นิยาย') not in atv_list):
      a_type = 'Education'     
    elif (text in ['ทำ','อบ','คั้น']) :
      if 'ขนม' in atv_list or 'อาหาร' in atv_list or 'กับข้าว' in atv_list or 'เย็น' in atv_list or 'เช้า' in atv_list or 'กลางวัน' in atv_list or 'น้ำส้ม' in atv_list:
        if 'ซื้อ' not in atv_list:
      #and (('ขนม' or 'อาหาร' or 'กับข้าว' or 'เย็น' or 'เช้า' or 'กลางวัน' ) in atv_list) and (('ซื้อ') not in atv_list):
          a_type = 'ทำอาหาร'
    elif (text in ['ทำอาหาร','ทำกับข้าว']) :
      if 'ซื้อ' not in atv_list:
        a_type = 'ทำอาหาร'      
    elif (text in ['วิ่ง','ฟิตเนส','โยคะ','fitness','ออกกำลังกาย','gym','จักรยาน','ว่ายน้ำ','execise']):
      if 'ชุด' not in atv_list or'เปลี่ยน' not in atv_list :
    #and (('ชุด'or'เปลี่ยน'or'อัด') not in atv_list):
        a_type = 'ออกกำลังกาย'
    elif (text in ['นอนหลับ','นอน','sleep','bed','เข้านอน','น็อก','หลับ']) :
      if 'ทำการบ้าน' not in atv_list and 'ดู' not in atv_list and 'อ่าน' not in atv_list and 'เล่น' not in atv_list and 'อ่านหนังสือ' not in atv_list and 'read' not in atv_list and 'checking' not in atv_list and 'อยาก' not in atv_list:
      #and (('ทำการบ้าน'or'ดู'or'อ่าน'or'เล่น'or'อ่านหนังสือ'or'read'or'checking' or 'อยาก') not in atv_list)
        a_type = 'นอน' 
    elif text in ['ไวน์','wine','เหล้า','กินเลี้ยง','party','งานเลี้ยง','wedding','งานแต่ง']:
      a_type = 'เข้าสังคม(Party)'  
    elif (text in ['ทำความสะอาด','เก็บ','งานบ้าน','รดน้ำต้นไม้','ล้างจาน','laundry','dry']) and (('requirement') not in atv_list):
      a_type = 'งานบ้านงานเรือน'     
    elif (text in ['นั่งสมาธิ','ทำบุญ','วัด','เข้าวัด','สวดมนต์','ตักบาตร']):
      a_type = 'ธรรมะ' 
    elif (text in ['board','football','park']) and (('เตรียม') not in atv_list) :
      a_type = 'กิจกรรม outdoor'      
    else :
      a_type = ''
   
    temp.append(a_type)

    try:
      #df = df.append(pd.DataFrame([data_list], columns=['ID','Time','Activity','Type','Pain','Gain']),ignore_index=True)
      if a_type is not '' : 
        data_list =  [row['ID'],row['Time'],text,a_type,row['Pain'],row['Gain']]
        df.loc[len(df)] = data_list 
    except : 
      None
    
  temp = list(set(temp))
  activity_type.append(temp)
activities_all['type'] = activity_type
df.drop_duplicates(subset =['ID','Time','Type'], keep = False, inplace = True)
df.to_excel("output.xlsx")
activities_all.to_excel("activities_all_output.xlsx")

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 764 entries, 0 to 1049
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        764 non-null    object
 1   Time      764 non-null    object
 2   Activity  764 non-null    object
 3   Type      764 non-null    object
 4   Pain      601 non-null    object
 5   Gain      656 non-null    object
dtypes: object(6)
memory usage: 41.8+ KB


In [66]:
top_activity = df.groupby('Type').count()
pain_ratio_list = []
gain_ratio_list = []
for index,row in top_activity.iterrows():
  #pain_ratio = row['Pain']/row['ID'] 
  #gain_ratio = row['Gain']/row['ID']
  pain_ratio_list.append(row['Pain']/row['ID'])
  gain_ratio_list.append(row['Gain']/row['ID'])
  
top_activity['pain_ratio'] = pain_ratio_list
top_activity['gain_ratio'] = gain_ratio_list

top_activity.sort_values(['ID','Pain'], ascending=[0, 1])

,ID,Time,Activity,Pain,Gain,pain_ratio,gain_ratio
Type,,,,,,,
ทำงาน,120,120,120,96,97,0.800000,0.808333
ทานอาหาร,110,110,110,96,99,0.872727,0.900000
อาบน้ำ,83,83,83,62,67,0.746988,0.807229
เดินทาง,80,80,80,67,71,0.837500,0.887500
Education,51,51,51,40,46,0.784314,0.901961
นอน,50,50,50,24,37,0.480000,0.740000
ตื่นนอน,50,50,50,47,34,0.940000,0.680000
Entertainment,29,29,29,27,29,0.931034,1.000000
แต่งตัว,25,25,25,17,19,0.680000,0.760000
